In [67]:
from xml.dom import minidom
import numpy as np
import geopy.distance
import pandas as pd
import xml.etree.ElementTree as ET
import uuid
import matplotlib.pyplot as plt


TRIPS_COMPLETE_PATH = "trips_completo.xml"

tree = ET.parse(TRIPS_COMPLETE_PATH)
trips = tree.getroot()

In [68]:
selected_trips = []
edges = {}

In [69]:
def load_nodes(path):
    nodes = []
    events_xml = minidom.parse(path)
    map_nodes_tags = events_xml.getElementsByTagName('node')
    for n in map_nodes_tags:
        nodes.append(int(n.getAttribute('id')))
    return nodes

def load_edges(path):
    map_xml = minidom.parse(path)
    map_links_tags = map_xml.getElementsByTagName('link')
    for n in map_links_tags:
        edges[int(n.getAttribute('id'))] = [
            int(n.getAttribute('from')),
            int(n.getAttribute('to'))
        ]


In [70]:
nodes = load_nodes('map_complete.xml')
load_edges('map_complete.xml')

In [71]:
def find_start_link(origin, destination):
    for k, e in edges.items():
        if (e[0] == origin):
            return k
    return -1

In [72]:
for trip in trips.iter('trip'):
    origin = int(trip.attrib['origin'])
    destination = int(trip.attrib['destination'])
    if (origin in nodes and destination in nodes):
        selected_trips.append(trip)

In [73]:
df = pd.DataFrame(columns=["name", "origin", "destination", "link_origin", "count", "start", "mode", "uuid"])

In [74]:
counter = 0
for s in selected_trips:
    origin = int(s.attrib['origin'])
    destination = int(s.attrib['destination'])
    link_origin = int(find_start_link(origin, destination))
    
    if (link_origin == -1):
        continue
    
    name = s.attrib['name']

    old_count = int(s.attrib['count'])
    start = int(s.attrib['start'])
    mode = s.attrib['mode']
    uuid_trip = 'a'
    df.loc[counter] = [name, origin, destination, link_origin, old_count, start, mode, uuid_trip]
    counter += 1

In [75]:
df = df[(df.start >= 28800) & (df.start <= 30000)]

In [76]:
df.start = df.start - 28800

In [77]:
final_df = pd.DataFrame(columns=["name", "origin", "destination", "link_origin", "count", "start", "mode", "uuid"])    

In [78]:
df.describe()

,name,origin,destination,link_origin,count,start,mode,uuid
count,50,50,50,50,50,50,50,50
unique,50,46,46,46,30,4,1,1
top,7736,303987727,165459105,2150,23,1,car,a
freq,1,4,2,4,8,43,50,50


In [79]:
counter = 0
for row in df.iterrows():
    access = row[1]
    repeat = access['count']
    for v in range(1, repeat):    
        final_df.loc[counter] = [
            counter,
            access['origin'],
            access['destination'],
            access['link_origin'],
            1,
            access['start'],
            access['mode'],
            uuid.uuid4()
        ]
        counter += 1

In [82]:
final_df.to_csv('trips-with-uuid.csv', header=False, sep=';', index=False)